In [6]:
import os
import json
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print("API key found")
else:
    print("API key not found")

system_prompt = "you weather telling ai assitant and not talk about anything else "

openai = OpenAI(api_key=openai_api_key)

weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Returns the weather info for a given city. Only use this to answer weather-related questions.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "Name of the city to get weather for"
                }
            },
            "required": ["city"]
        }
    }
}

weather_data = {
    "Lahore": "Cold, 16°C",
    "Karachi": "Hot, 32°C", 
    "Islamabad": "Mild, 22°C",
    "Peshawar": "Cool, 18°C",
    "Quetta": "Chilly, 12°C",
    "Multan": "Warm, 28°C"
}

def get_city_weather(city):
    city = city.title()  # ensures first letter capitalized
    return weather_data.get(city, "Sorry, weather info not found for this city.")

def chat(message, history):
    # Convert Gradio history format to OpenAI format
    messages = [{"role": "system", "content": system_prompt}]
    
    for user_msg, assistant_msg in history:
        messages.append({"role": "user", "content": user_msg})
        if assistant_msg:
            messages.append({"role": "assistant", "content": assistant_msg})
    
    messages.append({"role": "user", "content": message})
    
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        tools=[weather_tool],
        tool_choice="auto"
    )
    
    msg = response.choices[0].message
    
    # If the model wants to call the function
    if msg.tool_calls:
        tool_call = msg.tool_calls[0]
        if tool_call.function.name == "get_weather":
            args = json.loads(tool_call.function.arguments)
            city = args.get("city", "")
            weather_info = get_city_weather(city)
            return weather_info
    else:
        content = msg.content or ""
        return content

# Gradio interface
iface = gr.ChatInterface(fn=chat, title="Weather AI Assistant")
iface.launch()

C:\Users\PC\anaconda3\envs\llms\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


API key found
* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\PC\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\PC\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\PC\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2229, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\PC\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1738, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\PC\anaconda3\envs\llms\Lib\site-packages\gradio\utils.py", line 907, in async_wrapper
    response = await f(*args, **kwargs)
               ^^^^^^^^^^^^^^